 <img src="../images/attenuation.gif" width=500 alt="radar-gif"></img>

# Radares Meteorológicos

---

## Resumen
En este cuadernillo (Notebook) aprenderemos:

1. Breve introducción a los radares meterológicos
2. Red de Nacional de radares meteorológicos de Colombia
4. Consulta de datos usando el bucket de AWS de [radares](https://registry.opendata.aws/ideam-radares/) 
1. Generación de gráficas de reflectividad
1. Generación de campos de lluvia usando la ecuación de Marshall & Gunn (1953)

## Prerequisitos
| Conceptos | Importancia | Notas |
| --- | --- | --- |
| [Introducción a Xarray](https://foundations.projectpythia.org/core/xarray.html) | Necesario | lectura de datos multidimensionales |
| [Introducción a Py-Art](https://projectpythia.org/radar-cookbook/README.html) | Necesario | lectura de datos de radar |
| [Introducción a Xradar](https://docs.openradarscience.org/projects/xradar/en/stable/notebooks/Read-plot-Sigmet-data-from-AWS.html) | Necesario | lectura de datos de radar |
| [Introducción a Wradlib](https://docs.wradlib.org/en/latest/) | Necesario | lectura de datos de radar |
| [Fundamentos de radar meteorológico](https://www.meted.ucar.edu/radar/basic_wxradar_es/) | Útil | Fundametos básicos en radares meteorológicos |
| [Introducción a  NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Útil | Entender la metadata de los datos |

- **Tiempo de aprendizaje**: 30 minutos

## 1. Radares meteorológicos

Los radares meteorolófico son sensores activos que emiten pulsos de energia a la atmosfera y miden la energia retrodispera de todos los objetos a lo largo del haz de midición como se puede observar en la siguiente animación del curso Fundamentos en radares meterológicos (The Commet program - meted.ucar.edu)

In [ ]:
# @markdown

from IPython.display import IFrame
from ipywidgets import widgets
from IPython.display import HTML


HTML("""
    <video alt="test" controls>
        <source src="../images/radar_atten.mp4" type="video/mp4">
    </video>
""")

Tradicionalmente los radares meteorológicos operan en modo de Indicador de Posicion Plana (PPI por sus siglas en inglés de Plan Position Indicator). Este modo primero fija la elevacion de la antena a un ángulo determinado con respecto a la horizontal y gira 360 grados con respecto al norte. Luego repite esta tarea multiples veces a diferentes elevaciones hasta completar un Patron de Cobertura Volumétrico (VCP por sus siglas en inglés de Volume Coverage Pattern) como se puede obervar en la siguiente animación del curso Fundamentos en radares meterológicos (The Commet program - meted.ucar.edu)

In [ ]:
# @markdown

from IPython.display import HTML

HTML("""
    <video alt="test" controls>
        <source src="../images/radar_scan_EM_animation.mp4" type="video/mp4">
    </video>
""")

## 2. Red Nacional de Radares de Colombia

Colombia cuenta actualmente con una red de 12 radares meterológicos que hacen vigilancia continua a las condiciones de precipitación en el pais como se puede observar en la siguiente Tabla

| Entidad | Número de radares | banda | 
| --- | :---: | :---: |
| IDEAM  |  4  | C  |
| Aerocivil |  6  |  C  | 
| SIATA | 1  | C  |
| IDIGER | 1  | X  | 

### Librerias

A continuación vamos a importar las librerias que utilizaremos en este cuadernillo

In [ ]:
import pyart
import fsspec
import matplotlib.pyplot as plt
import xradar as xd
import folium
import numpy as np
import pandas as pd
import boto3
import botocore
import cartopy.crs as ccrs
from folium import plugins

### 2.1 Ubicación de los radares meteorológicos

Podemos usar `folium` para crear un mapa dinámico donde visualizamos la ubicación de los radares meteorológicos en Colombia

In [ ]:
min_lon, max_lon, min_lat, max_lat = -90, -72, -1, 14

map_ = folium.Map(location=[8, -76],
                  zoom_start = 6, 
                  min_lat=min_lat, 
                  max_lat=max_lat, 
                  min_lon=min_lon, 
                  max_lon=max_lon, 
                  zoom_control=False,
                  control_scale=True,
                  scrollWheelZoom=False,
                  width=1000,height=700)

folium.TileLayer('openstreetmap').add_to(map_)
folium.TileLayer('cartodbpositron').add_to(map_)
folium.TileLayer('stamenterrain').add_to(map_)
folium.TileLayer('cartodbdark_matter').add_to(map_)
folium.LayerControl().add_to(map_)

minimap = folium.plugins.MiniMap()
map_ = map_.add_child(minimap)
# map_

Ahora podemos agregar la ubicación de cada radar en el mapa

In [ ]:
def locate_radar(row):   
    df = row.to_frame()
    html = df.iloc[:-3].to_html(classes="table table-striped table-hover table-condensed table-responsive")

    popup = folium.Popup(html, max_width=2650)
    folium.Marker(location=[row.lat, row.lon], popup=popup).add_to(map_)
    
    rad = float(row['rad'])
    circle = folium.vector_layers.Circle(location=[row.lat, row.lon], color=row.color, fill=True, 
                                               fill_color=f"{row['color']}", 
                                              radius= rad * 2500, fill_opacity=0.1)     
    circle.add_to(map_)

In [ ]:
df_radares = pd.read_csv('../data/radar_locations.csv')

Aplicamos la función `locate_radar` a cada fila del dataframe usando el método `pd.apply` 

In [ ]:
df_radares.apply(locate_radar, axis=1)
map_

#### 2.2 Radares meteorológicos de IDEAM disponibles en [AWS](https://registry.opendata.aws/ideam-radares/)

El Instituto de Hidrología, Meteorología y Estudios Ambientales - IDEAM ha dispuesto de manera públicos los datos del radar meteorológico. Los datos se pueden encontrar [aquí](https://registry.opendata.aws/ideam-radares/).

La estructura del bucket es **s3://s3-radaresideam/l2_data/YYYY/MM/DD/Radar_name/RRRAAMMDDTTTTTT.RAWXXXX** donde: 

* **YYYY** es el año de 4 dígitos
* **MM** es el mes de 2 dígitos
* **DD** es el mes de 2 dígitos día
* **Radar_name** nombre del radar. Las opciones son Guaviare, Munchique, Barrancabermja y Carimagua
* **RRRAAMMDDTTTTTT** es el nombre del archivo del radar con lo siguiente:
  - **RRR** las tres primeras letras del nombre del radar (p. ej., GUA para el radar Guaviare)
  - **YY** es el año de 2 dígitos
  - **MM** es el mes de 2 dígitos
  - **DD** es el día de 2 dígitos
  - **TTTTTT** es la hora en la que se realizó el escaneo (GTM)
  - **RAWXXXX** Formato de archivo Sigmet y código único proporcionado por el software IRIS

Veamos un ejemplo de cómo luce el repositorio usando aws cli y el siguiente commando en la terminal ```aws s3 ls --no-sign-request s3://s3-radaresideam/```

In [ ]:
!aws s3 ls --no-sign-request s3://s3-radaresideam/l2_data/2021/09/19/

## 3. Acceso a los datos de radar usando Python

Podemos acceder a los datos de radar usando `Python` y librerias como `Py-Art` o `Xradar`. Para leer los datos de radar primero necesitamos generar una conexión entre el repositorio de datos `s3` de Amazon y nuestro computador. Esto lo podemos lograr mediante la libreria [`boto3`](https://aws.amazon.com/es/sdk-for-python/)

In [ ]:
# direccion del bucket
str_bucket = "s3://s3-radaresideam/"
# creacion de un objeto s3 
s3 = boto3.resource(
    "s3",
    config=botocore.client.Config(signature_version=botocore.UNSIGNED, user_agent_extra="Resource"),
)
# Creamos un objeto s3.Bucket
bucket = s3.Bucket("s3-radaresideam")
# Generamos una consulta tipo
query = 'l2_data/2022/08/09/Carimagua/CAR22080919'
# Listamos los archivos dentro de nuestro bucket
radar_files = [f"{str_bucket}{i.key}" for i in bucket.objects.filter(Prefix=f"{query}")]
radar_files[:5]

Esta forma es un poco complicada. Podemos utilizar [`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/) que permite hacer un "montaje" del bucket nuestro computador y es mucho mas simplificado. 

In [ ]:
fs = fsspec.filesystem("s3", anon=True)
files = sorted(fs.glob("s3-radaresideam/l2_data/2022/08/09/Carimagua/CAR22080919*"))
files

Una vez que tengamos nuestros archivos de interés, podemos cargar uno en memoria usando `Xradar` o `Py-Art`. Por ejemplo, estamos interesados ​​en el octavo archivo (`index=7`).


### 3.1 Lectura de datos usando [`Py-Art`](https://arm-doe.github.io/pyart-docs-travis/index.html)

Podemos leer nuestros datos usando el modulo `pyart.io.read_sigmet` o de manera general `pyart.io.read`

In [ ]:
radar = pyart.io.read_sigmet(f's3://{files[7]}')

Ahora podemos consultar la informacion contenida dentro del objeto radar usando `rada.info('compact')`

In [ ]:
radar.info('compact')

Podemos listar las variables polarimétricas contenidas en este archivo usando `radar.fields`

In [ ]:
list(radar.fields)

La variable polarimétrica más popular en es el factor de reflectividad que es una medida indirecta de la intensidad y el tamaño de las gotas de lluvia. Podemos ver mas información en el objeto radar accediendo de manera similar a un `diccionario`

In [ ]:
radar.fields['reflectivity']

#### Gráficas de reflectividad ($Z$)

Ahora podemos generar salidas gráficas usando el módulo [`pyart.graph`](https://arm-doe.github.io/pyart-docs-travis/API/generated/pyart.graph.html?highlight=graph#module-pyart.graph). Esta libreria nos permite realizar gráficos con o sin georeferenciacion usando [`pyart.graph.RadarDisplay`](https://arm-doe.github.io/pyart-docs-travis/API/generated/pyart.graph.RadarDisplay.html#pyart.graph.RadarDisplay) o [`pyart.graph.RadarMapDisplay`](https://arm-doe.github.io/pyart-docs-travis/API/generated/pyart.graph.RadarMapDisplay.html#pyart.graph.RadarMapDisplay)

In [ ]:
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(121)
display_ = pyart.graph.RadarDisplay(radar)
display_.plot(
    "reflectivity",
    0,
    ax=ax,
    vmin=0,
    vmax=60.0,
    colorbar_label="Reflectivity (dBZ)",
    title="Referencia al centro del radar",
)

projection = ccrs.PlateCarree()
ax1 = plt.subplot(122, projection=projection)
display_ = pyart.graph.RadarMapDisplay(radar)
# Extract the latitude and longitude of the radar and use it for the center of the map
lat_center = round(radar.latitude['data'][0], 0)
lon_center = round(radar.longitude['data'][0], 0)


# Determine the ticks
lat_ticks = np.arange(lat_center-3, lat_center+3, 1.5)
lon_ticks = np.arange(lon_center-3, lon_center+3, 1.5)

# Set the projection - in this case, we use a general PlateCarree projection

display_.plot_ppi_map("reflectivity", 0, 
                     resolution='10m', 
                     ax=ax1, 
                     lat_lines=lat_ticks, 
                     lon_lines=lon_ticks, 
                     colorbar_label="Reflectivity (dBZ)",
                     title="Georeferenciado",)

#### Gráficas de variables polarimétricas

Tambien podemos generar gráficas de otras variables como reflectividad diferencial ($Z_{DR}$), diferencial de fase ($\phi_{DP}$) y el coeficiente de correlación copolar ($\rho_{HV}$)

In [ ]:
fig = plt.figure(figsize=(12,10))
display_ = pyart.graph.RadarMapDisplay(radar)
# Extract the latitude and longitude of the radar and use it for the center of the map
lat_center = round(radar.latitude['data'][0], 0)
lon_center = round(radar.longitude['data'][0], 0)


# Determine the ticks
lat_ticks = np.arange(lat_center-3, lat_center+3, 1.5)
lon_ticks = np.arange(lon_center-3, lon_center+3, 1.5)

# Set the projection - in this case, we use a general PlateCarree projection
projection = ccrs.PlateCarree()
ax1 = plt.subplot(221, projection=projection)
display_.plot_ppi_map("reflectivity", 0, 
                     resolution='10m', 
                     ax=ax1, 
                     lat_lines=lat_ticks, 
                     lon_lines=lon_ticks, 
                     title="$Reflectividad$")

ax2 = plt.subplot(222, projection=projection)
display_.plot_ppi_map("differential_reflectivity", 0, 
                     resolution='10m', 
                     ax=ax2, 
                     lat_lines=lat_ticks, 
                     lon_lines=lon_ticks, 
                     title="$Reflectividad \  diferencial$")

ax3 = plt.subplot(223, projection=projection)
display_.plot_ppi_map("differential_phase", 0, 
                     ax=ax3, resolution='10m', 
                     lat_lines=lat_ticks, 
                     lon_lines=lon_ticks, 
                     title="$Diferencial \ de \ fase$")

ax4 = plt.subplot(224, projection=projection)
display_.plot_ppi_map("cross_correlation_ratio", 0, 
                     ax=ax4, resolution='10m', 
                     lat_lines=lat_ticks, 
                     lon_lines=lon_ticks, 
                     title="$Coeficiente \ de \ corralación \ copolar$")
plt.tight_layout()

### 3.2 Lectura de datos usando [`Xradar`](https://docs.openradarscience.org/projects/xradar/en/latest/index.html)

Los datos de radar de IDEAM también se puden leer usando [`xradar.io.open_iris_datatree`](https://docs.openradarscience.org/projects/xradar/en/stable/generated/xradar.io.backends.iris.open_iris_datatree.html#xradar.io.backends.iris.open_iris_datatree). Esta libreria, a diferencia de `Py-Art` usa como base [`Xarray`](https://docs.xarray.dev/en/stable/) retornando un [`xarray.Dataset`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) en caso de seleccionar un barrido en particular o un [`xarray.datatree`](https://xarray-datatree.readthedocs.io/en/latest/)  en caso de leer todas las elevaciones contenidas dentro del mismo archivo.

In [ ]:
## preaparamos un archivo para abrir de manera local
task_file = fsspec.open_local(f'simplecache::s3://{files[7]}',
                                s3={'anon': True},
                                filecache={'cache_storage': '.'})

In [ ]:
# leemos nuestro archivo usando xradar
radar = xd.io.open_iris_datatree(task_file)
display(radar)

Este `datatree` solo contine una elevación. Podemos acceder usando notación similar a la de un diccionario

In [ ]:
display(radar['sweep_0'])

#### Grafico de reflectividad

Podemos usar la funcionalidad de `xarray` para generar plots de la siguiente manera

In [ ]:
radar['sweep_0']['DBZH'].plot(cmap='pyart_ChaseSpectral', vmin=-10, vmax=60)

Como podemos observar en el gráfico anterior los datos se encuentran ordenados por las dimensiones y coordenadas `azimuth` y `range`. Podemos usar la `xr.georeference` para generar las coordenadas polares a nuestro `dataset`

In [ ]:
radar = radar.xradar.georeference()
display(radar['sweep_0'])

Como podemos ver `x`, `y` y `z` fueron agregados como nuevos campos de coordenadas. Ahora podemos proceder a generar el gráfico pero esta vez de manera georeferenciado

In [ ]:
radar['sweep_0']['DBZH'].plot(x='x', y='y', cmap='pyart_ChaseSpectral', vmin=-10, vmax=60)

Para generar gráficas de radar en un sistema coordenado de referencia podemos usar `cartopy`. Extraemos el sistema de referencia desde el objeto radar usando `xd.georeference.get_crs`

In [ ]:
proj_crs = xd.georeference.get_crs(radar["sweep_0"].ds)
cart_crs = ccrs.Projection(proj_crs)

Ahora sí procedemos a generar el gráfico de reflectividad en un sistema coordenado de referencia

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
radar["sweep_0"]["DBZH"].plot(
    x="x",
    y="y",
    cmap='pyart_ChaseSpectral',
    transform=cart_crs,
    cbar_kwargs=dict(pad=0.075, shrink=0.75), 
    vmin=-10, vmax=60)
ax.coastlines()
ax.gridlines(draw_labels=True, ls='--', lw=0.5)

#### Gráficos interactivos usando [`hvplot`](https://hvplot.holoviz.org/)

Los gráficos estáticos pueden ser usados principalmente para publicaciones donde la interactividad no es necesaria. Sin embargo, en cursos como el taller de programación de AtmosCol 2023 podemos utilizar las ventajas de la interactividad para obtener mayor provecho de la información. 

In [ ]:
import hvplot.xarray

In [ ]:
ref = radar["sweep_0"].DBZH.hvplot.quadmesh(x='x',
                                           y='y',
                                           cmap='pyart_ChaseSpectral',
                                           clabel='Horizontal Reflectivity (dBZ)',
                                           title=f'Horizontal Reflectivity \n {radar.attrs["instrument_name"]} Radar',
                                           clim=(-20, 60),
                                           height=500,
                                           rasterize=True,
                                           width=600,)
ref

Ahora podemos interacturar con mas de un plot a la vez

In [ ]:
zdr = radar["sweep_0"].ZDR.hvplot.quadmesh(x='x',
                                             y='y',
                                             cmap='pyart_ChaseSpectral',
                                             clabel='Differential Reflectivity (dB)',
                                             title=f'Horizontal Reflectivity \n {radar.attrs["instrument_name"]} Radar',
                                             clim=(-1, 6),
                                             height=500,
                                             rasterize=True,
                                             width=600,)

In [ ]:
(ref + zdr)

## 4. Estimados de lluvia utilizando la ecuación de [Marshall and Gunn (1953)](https://www.sciencedirect.com/science/article/abs/pii/S0065268708603106?via%3Dihub)

Uno de los productos de mayor interés para la comunidad científica, y en genera para todos los usuarios de la información de radares meteorológicos, es la **estimación de lluvia**. Desde **1948** los científicos han tratado de convertir las mediciones de **reflectividad ($Z$)** en mediciones de **intensidad de precipitación ($R$)**. Para esto se han desarrollado multiples ecuación empiricas **$Z-R$**. En 1953, Marshall and Gunn encontraron una de las relaciones más comunes e incluso utilizadas en la actualidad

**$$Z  = 200 R ^{1.6}$$**

Depajando para R tenemos:

$$R[mm hr^{-1}] = \left[ \frac{Z[mm^{6} mm^{-3}]}{200} \right] ^ {\frac{1}{1.6}}$$

$$R[mm hr^{-1}] = \left( \frac{1}{200} \right) ^ {\frac{1}{1.6}} \left( Z[mm^{6} mm^{-3}] \right) ^ {\frac{1}{1.6}}$$

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Cuidado!!!</p>
    La reflectividad del radar esta dada en unidades decibelicas es decir<br>
    $$Z [dBZ] = 10 * log_{10}(Z[mm^{6}m^{3}])$$ <br>
    Por lo tanto debemos convertir a unidades lineales<br>
    $$Z[mm^{6}m^{3}] = 10 ^{\left(\frac{Z[dBZ]}{10}\right)}$$
</div>

In [ ]:
# Reflectividad en unidades decibelicas
ref_log = radar["sweep_0"].DBZH
# reflectividad en unidades lineales
ref_lin = 10 ** (ref_log / 10)
#estimado de lluvia usando Marshall and Gunn (1953)
rr = (1 / 200) ** (1 / 1.6 ) * ref_lin ** (1 / 1.6)

Ahora podemos visualizar el campo de lluvia generado

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
rr.plot(
    x="x",
    y="y",
    cmap='jet',
    transform=cart_crs,
    cbar_kwargs=dict(pad=0.075, shrink=0.75, label='$Intensidad \ de \ lluvia \ [mm hr^{-1}]$'), 
    vmin=0, vmax=50)
ax.set_title("$Estimado \ de \ lluvia$")
ax.coastlines()
ax.gridlines(draw_labels=True, ls='--', lw=0.5)

---

## Conclusiones

En este cuadernillo aprendimos sobre la red nacional de radares meteorológicos de Colombia. Logramos acceder a los datos del repositorio de AWS usando Python. Utilizamos Py-Art y Xradar para acceder, visualizar los datos de manera intectiva usando hvplot y generar estimados de precipitación utilzando la ecuacion de Marshall and Gunn (1953)

## Resources and references

* Radar Cookbook [https://projectpythia.org/radar-cookbook/README.html] DOI[https://doi.org/10.5281/zenodo.8075855]
* Rose, B. E. J., Kent, J., Tyle, K., Clyne, J., Banihirwe, A., Camron, D., May, R., Grover, M., Ford, R. R., Paul, K., Morley, J., Eroglu, O., Kailyn, L., & Zacharias, A. (2023). Pythia Foundations (Version v2023.05.01) https://doi.org/10.5281/zenodo.7884572
* Marshall, J. S., and W. M. K. Palmer, 1948: THE DISTRIBUTION OF RAINDROPS WITH SIZE. J. Atmos. Sci., 5, 165–166, https://doi.org/10.1175/1520-0469(1948)005<0165:TDORWS>2.0.CO;2.
* Helmus, J.J. & Collis, S.M., (2016). The Python ARM Radar Toolkit (Py-ART), a Library for Working with Weather Radar Data in the Python Programming Language. Journal of Open Research Software. 4(1), p.e25. DOI: http://doi.org/10.5334/jors.119
* Xradar Python library [https://docs.openradarscience.org/projects/xradar/en/stable/#] DOI[https://docs.openradarscience.org/projects/xradar/en/stable/#]